<a href="https://colab.research.google.com/github/aminramezani345/BioGPT/blob/main/HCCNotes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#########Install requirements###########################

!pip install --upgrade transformers
!pip install langchain
!pip install --upgrade tokenizers
!pip install pyreadr
!pip install --upgrade accelerate
!pip install --upgrade transformers
!pip install langchain
!pip install --upgrade tokenizers
!pip install pyreadr
!pip install --upgrade accelerate
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import re
from langchain.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain.prompts import PromptTemplate
from time import time
import os
import pandas as pd
import pyreadr
from collections import OrderedDict
import pandas as pd


In [ ]:


text = """

"APHDR    MEDICAL RECORD |                   SURGICAL PATHOLOGY
Specimen (Received Aug 06, 2008 11:45):
1)GALLBLADDER
2)LIVER WEDGE RESECTION RIGHT
BRIEF CLINICAL HISTORY:
     66YO WITH MAJOR PSYCH HX, ASTHMA, HTN
PREOPERATIVE DIAGNOSIS:
    HCC
OPERATIVE FINDINGS:
POSTOPERATIVE DIAGNOSIS:
     HCC
                           Surgeon/physician:Amin Eydi Ramezani

=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=

- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

                              PATHOLOGY REPORT
Laboratory:                        Accession No.

Microscopic exam/diagnosis:

     Specimen:  1.  Gallbladder.  2.  Liver wedge resection, right.

     Clinical History:  66-year-old with major psych history, asthma,

     hypertension.  Per surgeon and clinical notes, the patient is status

     post chemoembolization (on 6/19/08).  Liver mass found on 5/15/08.
    Operation and Findings:  Right hepatectomy, general anesthesia,
   central line/CVP.  HCC.
    Gross Description:  Case blablabla  Two specimens are received fresh from the OR, labeled with the
    patient name, Douglass Lawsone. The parenchymal margin is inked,
    bread-loafed and then showed surgeon.
    The first specimen is received in formalin, additionally labeled
   GALLBLADDER and contains a gallbladder that measures 7.5 x 4.0 x 4.0
   cm.  The serosal surface is pink-red, smooth and glistening.  The
    adventitial surface is slightly denuded, pink-red.  The overall
   surface appears edematous and congested.  The specimen has been opened
   to reveal normal gallbladder parenchyma near the cystic duct opening.
   The average wall thickness measures 0.1 cm.  The area measuring at
   least 1 cm away from the cystic duct opening has a wall thickness of
     approximately 1.0 cm and there is a 2.0 x 1.0 x 1.0 cm yellow,
    well-circumscribed nodule.  Representative sections of the nodule and
    surrounding thickened wall are submitted in cassettes A1-A2 and
   representative sections near the cystic duct are submitted in cassette
    A3.  Fujiwara for Surg Path/bll/8/7/08
    The second specimen is inked on the surgical margin and bread-loafed
   and then shown to the surgeon for OR consult.  The specimen measures
   18.5 x 11.9 x 4.5 cm and consists of a LIVER LOBE.  The specimen is
  serially sectioned to reveal a 12.2 x 4.4 x 2.9 cm yellow nodule,
    which abuts the serosal surface and is 0.4 cm from the black surgical
   margin at its closest.  The first yellow nodule extends for about 4.0
   cm diagonally into a second nodule, which measures 3.0 x 1.2 cm.
    There is an additional white nodule that measures 2.4 x 1.2 cm and is
   3.2 cm from the nearest black surgical margin.  A metal stent is
    identified inside one of the intraparenchymal vessels.  Representative
   sections are submitted as follows:  B1 is the section of the largest
   tumor closest to the black surgical margin.  B2 and B3 are
  representative sections of the yellow nodule.  B4 and B5 are
    representative sections of connection between the yellow nodules.  B6
  and B7 are representative sections of the second yellow nodule.  B8
   and B9 are representative sections of the third white nodule.  B10 is
   a representative section of non-neoplastic liver.  Jennings for Surg
   Path/bll/8/6/08
  Comment:  In addition to the larger necrotic area described below, a
   few small foci of viable hepatocellular carcinoma were found in
  representative sections, one of which is 2 mm from the inked
  parenchymal margin (slide B11).  There are extensive necrotic areas
    containing larger nodules that appear to have atypical architecture,
   which are presumed to represent necrotic carcinoma; however, other
   necrotic areas have unclear features.  Due to extensive necrosis in
   the liver specimen, the actual size of the tumor cannot be assessed in
  the tissue, but the largest atypical necrotic nodule is approximately
   3 cm in size.  TNM staging was done based on size reported in the
   radiology report which reports a 3.1 x 2.6 cm ill-defined
   hypervascular lesion in the right hepatic lobe (CT abdomen, 5/15/08,
   read by  Lee/Nino-Murcia).  Reviewed by:
   DIAGNOSIS:
    GALLBLADDER, CHOLECYSTECTOMY
    --  CONSISTENT WITH INFARCTED GALLBLADDER WALL WITH ACUTE
   INFLAMMATION
    LIVER, RIGHT WEDGE RESECTION
      --  MULTIFOCAL HEPATOCELLULAR CARCINOMA (SEE COMMENT)
      --  MICRONODULAR CIRRHOSIS
    Specimen Type: Liver wedge resection
   Focality: Multifocal
   Tumor Size: SEE COMMENT
    Histologic Type: Hepatocellular carcinoma
     Histologic Grade: Grade 1, well differentiated
     Lymph nodes: Not submitted
     Margins: 2 mm from parenchymal margin
     T2 NX MX
 """


In [5]:
cleaned_text = re.sub(r'[^\w\s:.]', '', text)

cleaned_text = re.sub(r'\n\s*\n', '\n', cleaned_text)

cleaned_text = re.sub(r':', ' :', cleaned_text)

cleaned_text = re.sub(r'\n\s*\n', '\n', cleaned_text)


In [1]:
OUT = pd.DataFrame(columns=['filename', 'Tumor Dimension','Closest Mrgine'])


NameError: ignored

In [ ]:
model_id = "Upstage/SOLAR-10.7B-Instruct-v1.0"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, temperature = 1, max_new_tokens=500,do_sample=True)
hf = HuggingFacePipeline(pipeline=pipe)
template = """{question}"""
prompt = PromptTemplate.from_template(template)
chain = prompt | hf

In [ ]:

#########Main###########################

st = time();
question=""" We are interested in extract information about specimens in following PATHOLOGY REPORT  that represent resected liver tissue. Some common relevant specimen names for resected liver tissue include:liver tissue, liver, liver resection, segment 1-8 or any permutation of 1-8, e.g. segment 2/3, wedge/partial/ extended/right/left/partial/wedge/formal/major resection, extended/right/left/partial/wedge/formal/major hepatectomy,
             liver/hepatic/anterior/posterior/partial segmentectomy, liver/hepatic/anterior/posterior/partial sectorectomy. Based on these permutations, examples may include “left lateral segmentectomy”,“extended right hepatectomy”, “posterior sectorectomy”, “”segmentectomy”, “segment 7/8 resection”, “left liver lobe”, “right liver lobe”, “partial liver resection”,
            “hepatic wedge resection”. The pathologist will often use multiple synonyms to denote the word “tumor”,which may include the following:"lesion","mass", "tissue abnormality","tumor",cancer","carcinoma","nodule",“focus”,“malignancy”.in the following pathology report :\n"""+ cleaned_text +"""the resected liver tissue dimensions is:"""
Tumor_Dimension=chain.invoke({"question": question})
question="""Only Find the the closest margin in the following pathology report :\n"""+ cleaned_text +"""."""
Closest_Margine=chain.invoke({"question": question})
OUT = OUT.append({'filename': file_name, 'Tumor Dimension': Tumor_Dimension,'Closest Mrgine':Closest_Margine}, ignore_index=True)
end = time()

print('****Time taken for the promp reply: ' + str(end-st))

tokenizer_config.json:   0%|          | 0.00/1.41k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/35.8k [00:00<?, ?B/s]

...

model-00002-of-00005.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00005.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00005.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00005-of-00005.safetensors:   0%|          | 0.00/1.69G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]